In [1]:
import os
import time
import wandb
import pickle
import torch
import faiss
import numpy as np
import pandas as pd
from typing import List, Dict, Any
import sys
sys.path.append('/content/drive/Othercomputers/kyei_mac_m3_pro/MediMaven')
from zenml import step, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from src.utils import get_mongo_connection
from sentence_transformers import SentenceTransformer


INFO:datasets:PyTorch version 2.6.0+cu124 available.


PyTorch version 2.6.0+cu124 available.


INFO:datasets:Polars version 1.21.0 available.


Polars version 1.21.0 available.


INFO:datasets:Duckdb version 1.2.1 available.


Duckdb version 1.2.1 available.


INFO:datasets:TensorFlow version 2.18.0 available.


TensorFlow version 2.18.0 available.


INFO:datasets:JAX version 0.5.2 available.


JAX version 0.5.2 available.


In [2]:
# db = get_mongo_connection()
# src_collection = 'qa_master_processed'
# collection = db[src_collection]
# data = list(collection.find({},{'_id' : 0}))
# df = pd.DataFrame(data)

# if "context_id" not in df.columns:
#     df["context_id"] = df.index.astype(str) 

# df

In [3]:
# meta_df = df[["context_id", "context"]]
# meta_df

In [2]:
querry = "what causes hypertension?"

## 1. Retriever

In [3]:
embed_mode_name = "sentence-transformers/all-MiniLM-L6-v2"

In [4]:
embeder = SentenceTransformer(embed_mode_name)
querry_embedding = embeder.encode(querry)
querry_embedding

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Use pytorch device_name: cuda


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([-1.07108587e-02,  6.39325529e-02, -2.17748825e-02,  6.37408569e-02,
        5.10996468e-02,  7.54497433e-03,  2.40731072e-02,  2.48579197e-02,
       -3.72231826e-02, -2.40275431e-02, -5.84572926e-03, -5.30485325e-02,
       -1.95338745e-02, -2.40437631e-02, -4.03857604e-02, -9.75393690e-03,
       -7.33973160e-02, -3.04172784e-02,  3.00706290e-02,  2.97163595e-02,
        3.17584798e-02,  5.93150370e-02, -4.10247855e-02,  3.11656054e-02,
       -1.98439024e-02,  3.16797160e-02,  3.24705429e-02, -8.07417557e-02,
       -9.29016061e-03, -1.42471045e-02,  2.60793008e-02, -2.27623917e-02,
       -4.33903225e-02,  6.57521039e-02,  1.17193498e-02,  7.83134848e-02,
        1.36830900e-02, -1.94479227e-02, -7.36206770e-02,  5.69710247e-02,
       -1.54986780e-03, -9.11833197e-02, -7.45328888e-02,  5.94863519e-02,
        3.20203118e-02,  5.00806533e-02, -4.46898639e-02,  5.12904525e-02,
        3.00790630e-02,  5.31335399e-02, -9.88918096e-02,  5.96192852e-03,
       -2.72185006e-03, -

## 2.FAISS INDEX

In [5]:
faiss_model_path =  "../models/faiss_index.bin"
retriever = faiss.read_index(faiss_model_path)


In [6]:
import json
with open("../models/faiss_index_data.json", "rb") as f:
    docs_metadata = json.load(f) 
docs_metadata

[{'context_id': '0',
  'faiss_id': 0,
  'context': 'alymphoid cystic thymic dysgenesis,congenital alopecia and nail dystrophy associated with severe functional T cell immunodeficiency,Pignata Guarino syndrome,winged helix deficiency T cell immunodeficiency, congenital alopecia, and nail dystrophy What is are T cell immunodeficiency, congenital alopecia, and nail dystrophy ?'},
 {'context_id': '1',
  'faiss_id': 1,
  'context': 'alymphoid cystic thymic dysgenesis,congenital alopecia and nail dystrophy associated with severe functional T cell immunodeficiency,Pignata Guarino syndrome,winged helix deficiency T cell immunodeficiency, congenital alopecia, and nail dystrophy How many people are affected by T cell immunodeficiency, congenital alopecia, and nail dystrophy ?'},
 {'context_id': '2',
  'faiss_id': 2,
  'context': 'alymphoid cystic thymic dysgenesis,congenital alopecia and nail dystrophy associated with severe functional T cell immunodeficiency,Pignata Guarino syndrome,winged heli

In [7]:
os.path.getsize("../models/faiss_index_data.json")/ (1024 * 1024)

44.09269142150879

In [8]:
retriever.search(np.expand_dims(querry_embedding, 0), 5)

(array([[0.62179816, 0.62611115, 0.65386   , 0.6567941 , 0.67504793]],
       dtype=float32),
 array([[11935,  4608,  7999, 13350, 16588]]))

In [9]:
### Test the retriever
dist, idx = retriever.search(np.expand_dims(querry_embedding, 0), 5)
docs = []
for d, i in zip(dist[0], idx[0]):
   doc = { 'doc_id': i,
         "context_id": docs_metadata[i]["context_id"],
         "score": d,
         "context": docs_metadata[i]["context"]}
   docs.append(doc)
docs

[{'doc_id': np.int64(11935),
  'context_id': '11211',
  'score': np.float32(0.62179816),
  'context': 'Hypertension, commonly known as high blood pressure, is a condition where the force of the blood against the artery walls is consistently too high. Hello doctor,\nMy friend is 40 years old. He was diagnosed with hypertension recently. What are the risk factors of this hypertension? What are the adverse health effects faced due to hypertension? What is the importance of regular blood pressure monitoring? What complications occur due to uncontrolled hypertension and how to manage it?\nPlease help.\nThank you.'},
 {'doc_id': np.int64(4608),
  'context_id': '4579',
  'score': np.float32(0.62611115),
  'context': 'Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,\nI am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing f

In [10]:
# Sort docs based on score in descending order 
sorted_docs = sorted(docs, key=lambda x: x['score'], reverse=True)
sorted_docs

[{'doc_id': np.int64(16588),
  'context_id': '15426',
  'score': np.float32(0.67504793),
  'context': 'Hypertension, commonly known as high blood pressure, is a medical condition characterized by elevated pressure in the arteries. Hello doctor,\nI am a 34 year old male diagnosed with hypertension. Kindly guide.\nThank you in advance.'},
 {'doc_id': np.int64(13350),
  'context_id': '12502',
  'score': np.float32(0.6567941),
  'context': 'High blood pressure, or hypertension, is a medical condition characterized by elevated pressure in the arteries. Hello doctor,\nI am a 28 year old woman, and with World Hypertension Day around the corner, I am eager to delve into blood pressure and its impact on overall health. Could you shed light on the factors influencing high blood pressure and the risks associated with unmanaged hypertension? I have heard lifestyle aspects like diet, exercise, and stress management are crucial, but I am keen on understanding their underlying mechanisms. Also, could

## 3.

## 4. Ranker

In [11]:
from src.utils import (get_mongo_connection, ensure_mongodb_running,
          cosine_similarity,l2_distance, compute_ndcg_at_k )


In [12]:
ranker_path = "../models/ltr_best_model.json" 
import xgboost as xgb

ranker = xgb.XGBRanker()
ranker.load_model(ranker_path)

In [13]:
cosine_similarity(querry_embedding, embeder.encode(docs[0]["context"]))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

np.float32(0.6891009)

In [14]:
docs_embeddings = [embeder.encode(doc['context']) for doc in docs]
cosims = [cosine_similarity(querry_embedding, doc_emb) for doc_emb in docs_embeddings]
l2s = [l2_distance(querry_embedding, doc_emb) for doc_emb in docs_embeddings]
lens = [len(doc['context'].split()) for doc in docs]
features = np.array([cosims, l2s, lens]).T
features


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[  0.68910092,   0.78854179,  81.        ],
       [  0.68694431,   0.79127198, 110.        ],
       [  0.67306995,   0.80861616,  75.        ],
       [  0.6716029 ,   0.81042838, 105.        ],
       [  0.66247606,   0.82161295,  36.        ]])

In [15]:
scores = ranker.predict(np.array([cosims, l2s, lens]).T)
scores

array([4.376121 , 4.9730306, 4.424193 , 4.8093796, 4.855015 ],
      dtype=float32)

In [16]:
for doc, score in zip(docs, scores):
    doc["ltr_score"] = float(score)

reranked_docs= sorted(docs, key=lambda x: x['ltr_score'], reverse=True)
reranked_docs

[{'doc_id': np.int64(4608),
  'context_id': '4579',
  'score': np.float32(0.62611115),
  'context': 'Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,\nI am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing factors of hypertension. I am interested in receiving guidance on lifestyle modifications that can help manage and control hypertension. I would like to know about how to monitor my blood pressure at home and a schedule for follow up appointments to track my progress. I would also like to know some preventive measures that might help maintain my overall cardiovascular health. Please help.\nThank you.',
  'ltr_score': 4.9730305671691895},
 {'doc_id': np.int64(16588),
  'context_id': '15426',
  'score': np.float32(0.67504793),
  'context': 'Hypertension, commonly known as high blood pressure, is a medical condition

## 4.Generate

In [17]:
# del model
# del tokenizer

# del prompt

# torch.cuda.empty_cache()

In [18]:
model_path ="../models/llama_finetuned/merged_4bit"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
# model = torch.compile(
#     model,
#     mode='max-autotune',    # Better than reduce-overhead, minimal accuracy impact
#     fullgraph=True,         # Enable full graph optimization
#     dynamic=False,          # Static shapes for better optimization
# )
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [20]:
# Get quantization config
config = model.config
print("Model configuration:")
print(f"Model dtype: {model.dtype}")
print(f"Model device: {model.device}")
print("\nConfig details:")
for key, value in config.to_dict().items():
    if 'quant' in key.lower():
        print(f"{key}: {value}")

Model configuration:
Model dtype: torch.float16
Model device: cuda:0

Config details:
quantization_config: {'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>, '_load_in_8bit': False, '_load_in_4bit': True, 'llm_int8_threshold': 6.0, 'llm_int8_skip_modules': ['lm_head'], 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'load_in_4bit': True, 'load_in_8bit': False}


In [21]:
print(f"{torch.cuda.memory_allocated() / 1e9:.2f} GB allocated on GPU")
print(f"{torch.cuda.memory_reserved() / 1e9:.2f} GB reserved on GPU")
print(f"{torch.cuda.get_device_name()}")
print(f"{torch.cuda.is_available()=}")
print(f"{torch.cuda.current_device()=}")

# Check if the model is on the GPU
if next(model.parameters()).is_cuda:
    print("Model is on GPU")
else:
    print("Model is NOT on GPU")

5.80 GB allocated on GPU
5.81 GB reserved on GPU
Tesla T4
torch.cuda.is_available()=True
torch.cuda.current_device()=0
Model is on GPU


In [22]:
context = "\n\n".join(d["context"] for d in reranked_docs[:3])
print("Context for generation:")
print(context)


Context for generation:
Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,
I am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing factors of hypertension. I am interested in receiving guidance on lifestyle modifications that can help manage and control hypertension. I would like to know about how to monitor my blood pressure at home and a schedule for follow up appointments to track my progress. I would also like to know some preventive measures that might help maintain my overall cardiovascular health. Please help.
Thank you.

Hypertension, commonly known as high blood pressure, is a medical condition characterized by elevated pressure in the arteries. Hello doctor,
I am a 34 year old male diagnosed with hypertension. Kindly guide.
Thank you in advance.

High blood pressure, or hypertension, is a medical condition c

In [23]:
def build_prompt(querry: str, context: str) -> str:
    sys_msg = (
        "You are a helpful medical AI assistant. "
        "Provide clear and concise answers based on the given question and context."
        "the answer is after the [/INST] tag so generate and summerize evrything after the [/INST] tag"
    )

    # Example format for Llama instructions:
    prompt = (
        f"[INST] <<SYS>>\n{sys_msg}\n<<SYS>>\n"
        f"Question: {querry}\n"
        f"Context: {context}\n"
        "[/INST]"
    )
    return prompt
prompt = build_prompt(querry, context)
print(prompt)


[INST] <<SYS>>
You are a helpful medical AI assistant. Provide clear and concise answers based on the given question and context.the answer is after the [/INST] tag so generate and summerize evrything after the [/INST] tag
<<SYS>>
Question: what causes hypertension?
Context: Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,
I am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing factors of hypertension. I am interested in receiving guidance on lifestyle modifications that can help manage and control hypertension. I would like to know about how to monitor my blood pressure at home and a schedule for follow up appointments to track my progress. I would also like to know some preventive measures that might help maintain my overall cardiovascular health. Please help.
Thank you.

Hypertension, commonly known as high blood

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device     .to(model.device)

In [25]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)
inputs["input_ids"].shape

torch.Size([1, 367])

In [26]:
output = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [27]:
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

[INST] <<SYS>>
You are a helpful medical AI assistant. Provide clear and concise answers based on the given question and context.the answer is after the [/INST] tag so generate and summerize evrything after the [/INST] tag
<<SYS>>
Question: what causes hypertension?
Context: Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,
I am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing factors of hypertension. I am interested in receiving guidance on lifestyle modifications that can help manage and control hypertension. I would like to know about how to monitor my blood pressure at home and a schedule for follow up appointments to track my progress. I would also like to know some preventive measures that might help maintain my overall cardiovascular health. Please help.
Thank you.

Hypertension, commonly known as high blood

In [38]:
from IPython.display import display, Markdown


In [28]:
print(response.split("[/INST]")[-1].strip())

The causes and contributing factors of hypertension include:
1. Genetics: Individuals with a family history of hypertension are more likely to develop the condition.
2. Age: Hypertension risk increases with age, especially after the age of 40.
3. Obesity: Being overweight or obese can lead to high blood pressure due to the increased strain on the blood vessels.
4. Lack of physical activity: Sedentary lifestyle contributes to hypertension by promoting weight gain, insulin resistance, and other cardiovascular risk factors.
5. Excessive salt intake: Consuming high amounts of sodium can increase blood pressure levels.
6. Stress: Chronic stress can lead to hypertension by causing the release of stress hormones, such as adrenaline and cortisol, which can raise blood pressure.
7. Sleep apnea: This condition can cause frequent awakenings and disruptions in sleep patterns, leading to increased blood pressure.
8. Kidney disease: Kidney problems can lead to hypertension by causing the kidneys to 

## Furtherr Optimization with GPTQModel

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from gptqmodel import GPTQModel



INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


In [19]:
model_dir = "../models/llama_finetuned/merged_4bit"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", torch_dtype="auto")

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set max_memory in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
from gptqmodel import GPTQModel, QuantizeConfig


In [23]:

quant_config = QuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    sym=False
)

4. Quantize the Model
Proceed with the quantization process:

In [ ]:
from gptqmodel import GPTQModel

quantizer = GPTQModel(
    model=model,
    tokenizer=tokenizer,
    quant_config=quant_config
)

quantized_model = quantizer.quantize()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: GPTQModel.__init__() got an unexpected keyword argument 'model'

: 

In [43]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


CUDA extension not installed.


CUDA extension not installed.


In [ ]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_path = "../models/llama_finetuned/merged_4bit"
tokenizer_path = "../models/llama_finetuned"
output_dir = "../models/llama_finetuned/autogptq"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True, local_files_only=True)

# Quantization configuration
quantization_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    sym=False
)

# Load and quantize the model
model = AutoGPTQForCausalLM.from_pretrained(
    model_path,
    quantize_config=quantization_config,
    trust_remote_code=True,
    local_files_only=True,            # ✅ ensures local model loading
)

# Save the quantized model
model.save_pretrained(
    output_dir,
    use_safetensors=True
)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:20                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/auto_gptq/modeling/auto.py:76 in from_pretrained         │
│                                                                                                  │
│    73 │   │   low_cpu_mem_usage: bool = False,                                                   │
│    74 │   │   use_triton: bool = False,                                                          │
│    75 │   │   inject_fused_attention: bool = True,                                               │
│ ❱  76 │   │   inject_fused_mlp: bool = True,                                                     │
│    77 │   │   use_cuda_fp16: bool = True,                                                        │
│    78 │   │   quantize_config: Optional[BaseQuantizeConfig] = None,                              │
│    79 │   │   model_basename: Optional[str] = None,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/auto_gptq/modeling/_base.py:787 in from_pretrained       │
│                                                                                                  │
│   784 │   │   else:  # remote                                                                    │
│   785 │   │   │   for ext in extensions:                                                         │
│   786 │   │   │   │   resolved_archive_file = cached_file(model_name_or_path, model_basename +   │
│ ❱ 787 │   │   │   │   if resolved_archive_file is not None:                                      │
│   788 │   │   │   │   │   break                                                                  │
│   789 │   │                                                                                      │
│   790 │   │   if resolved_archive_file is None: # Could not find a model file to use             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:573 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   570 │   │   │   )                                                                              │
│   571 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   572 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 573 │   │   │   return model_class.from_pretrained(                                            │
│   574 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   575 │   │   │   )                                                                              │
│   576 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:272 in _wrapper           │
│                                                                                                  │
│    269 │   def _wrapper(*args, **kwargs):                                                        │
│    270 │   │   old_dtype = torch.get_default_dtype()                                             │
│    271 │   │   try:                                                                              │
│ ❱  272 │   │   │   return func(*args, **kwargs)            

In [ ]:
from auto_gptq import AutoGPTQForCausalLM
model = AutoGPTQForCausalLM.from_quantized(
    "./models/llama_finetuned/autogptq",
    trust_remote_code=True
)

  Cloning https://github.com/PanQiWei/AutoGPTQ.git to /tmp/pip-req-build-y72kkgbk
  Running command git clone --filter=blob:none --quiet https://github.com/PanQiWei/AutoGPTQ.git /tmp/pip-req-build-y72kkgbk
  Resolved https://github.com/PanQiWei/AutoGPTQ.git to commit b52f2719ac31bc9e5ced2b0f2f8a129e1d621bb3
  Preparing metadata (setup.py) ... done
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error
  ERROR: Failed building wheel for auto_gptq
  Running setup.py clean for auto_gptq
Failed to build auto_gptq
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (auto_gptq)
